In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install -U keras-tuner

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
# from keras.utils import np_utils
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.applications import DenseNet201, ResNet101V2, InceptionV3, EfficientNetB4, Xception
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.preprocessing import image
from random import shuffle
import os, os.path
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.utils import class_weight
import cv2
# import keras_tuner as kt

In [ ]:
image_dest_dir = "/content/drive/MyDrive/SPECT_MPI_Dataset"

In [ ]:
dir_list_1 = (os.listdir(image_dest_dir))
dir_list_1

In [ ]:
valid_dir_path = image_dest_dir+'/'+dir_list_1[2]
test_dir_path = image_dest_dir+'/'+dir_list_1[0]
train_dir_path = image_dest_dir+'/'+dir_list_1[1]

In [ ]:
os.listdir(train_dir_path)

In [ ]:
for i in range(len(dir_list_1)):
    dir_list_2 = os.listdir(image_dest_dir+'/'+dir_list_1[i])
print(dir_list_2)


In [ ]:
print("\t\tTrain DataSet")
print("CT scans with normal            : " + str(len(os.listdir(train_dir_path+'/'+dir_list_2[1]))))
print("CT scans with abnormal          : " + str(len(os.listdir(train_dir_path+'/'+dir_list_2[0]))))

print("\n\t\tTest DataSet")
print("CT scans with normal            : " + str(len(os.listdir(test_dir_path+'/'+dir_list_2[1]))))
print("CT scans with abnormal          : " + str(len(os.listdir(test_dir_path+'/'+dir_list_2[0]))))

print("\n\t\tvalid DataSet")
print("CT scans with normal            : " + str(len(os.listdir(valid_dir_path+'/'+dir_list_2[1]))))
print("CT scans with abnormal          : " + str(len(os.listdir(valid_dir_path+'/'+dir_list_2[0]))))


In [ ]:
# train_image_list = os.listdir(train_dir_path+'/'+dir_list_2[1])

# w=10
# h=10
# fig=plt.figure(figsize=(15 , 20))
# columns = 5
# rows = 10
# for i in range(columns*rows):
#     path = train_dir_path +'/'+dir_list_2[1]+"/"+train_image_list[i]
#     img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
#     fig.add_subplot(rows, columns, i+1)
#     plt.imshow(img, cmap="gray")
# plt.show()

In [ ]:
# train_image_list = os.listdir(train_dir_path+'/'+dir_list_2[1])

# w=10
# h=10
# fig=plt.figure(figsize=(15 , 20))
# columns = 5
# rows = 10
# for i in range(1, columns*rows+1):
#     path = train_dir_path+'/'+dir_list_2[1]+'/' + train_image_list[i-1]
#     img = cv2.imread(path, cv2.IMREAD_COLOR)
#     fig.add_subplot(rows, columns, i)
#     plt.imshow(img)
# plt.show()

In [ ]:
# train_image_list = os.listdir(train_dir_path+'/'+dir_list_2[1])

# w=10
# h=10
# fig=plt.figure(figsize=(15 , 20))
# columns = 5
# rows = 10
# for i in range(1, columns*rows +1):
#     path = train_dir_path+'/'+dir_list_2[1]+"/" + train_image_list[i-1]
#     img = cv2.imread(path, cv2.IMREAD_COLOR)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#     fig.add_subplot(rows, columns, i)
#     plt.imshow(img)
# plt.show()

In [ ]:
image_width = 256
image_height = 256
if K.image_data_format() == 'channels_first':
    input_shape = (3,  image_height, image_width,)
else:
    input_shape = (image_height, image_width, 3)
input_shape

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0, # this performs min-max normalization
    shear_range=0.20,
    horizontal_flip=True,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2,
    height_shift_range=0.2,
    fill_mode='constant'
)

In [ ]:
test_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0, # this performs min-max normalization
)

In [ ]:
train_generator = train_datagen.flow_from_directory(
        train_dir_path,
        target_size=(image_height, image_width),
        batch_size=8,
        color_mode="rgb",
        class_mode='binary',
        shuffle=True,
)

In [ ]:
class_weights = class_weight.compute_class_weight(
                class_weight = 'balanced',
                classes = np.unique(train_generator.classes),
                y=train_generator.classes)
for cls in train_generator.classes:
    class_weights[cls]
class_weights_1 = { 0 : class_weights[0] , 1 : class_weights[1] }

In [ ]:
class_weights_1

In [ ]:
validation_generator = test_datagen.flow_from_directory(
        valid_dir_path,
        target_size=(image_height, image_width),
        batch_size=8,
        color_mode="rgb",
        class_mode='binary',
        shuffle=True,
)

In [ ]:
def sensitivity(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def specificity(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1-y_true) * (1-y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1-y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def fmed(y_true, y_pred):
    spec = specificity(y_true, y_pred)
    sens = sensitivity(y_true, y_pred)
    fmed = 2 * (spec * sens)/(spec+sens+K.epsilon())
    return fmed

def f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

In [ ]:
def macro_f1(y_true, y_pred):
     return f1_score(y_true, y_pred, average='macro')

**Importing pre-trained models**

In [ ]:
base_model_densenet = DenseNet201(weights='imagenet', include_top=False, input_shape=input_shape)

**Model Initialization**

In [ ]:
# base_model_densenet.summary()

In [ ]:
def generate_model():
    model=Sequential([
      base_model_densenet,
      Dense(167, activation='selu'),
      Dropout(0.9),
      Dense(157, activation='leaky_relu'),
      Dropout(0.2),
      Dense(191, activation='relu'),
      Dropout(0.6),
      Dense(268, activation='elu'),
      Dropout(0.8),
      Dense(40, activation='elu'),
      Dense(388, activation='relu'),
      Dropout(0.5),
      Dense(289, activation='leaky_relu'),
      Dropout(0.9),
      Dense(37, activation='leaky_relu'),
      Dropout(0.5),
      Dense(91, activation='selu'),
      Dropout(0.1),
      Dense(185, activation='selu'),
      Dropout(0.2),
      Dense(123, activation='selu'),
      Dropout(0.7),
      Dense(149, activation='selu'),
      Dropout(0.3),
      Flatten(),
      Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy', sensitivity, specificity, fmed, f1])
    return model

In [ ]:
model_densenet=generate_model()

In [ ]:
model_densenet.summary()

In [ ]:
num_train_samples = 128
num_validation_samples = 32
batch_size = 8

**Training Densenet**

In [ ]:
history_val_fmed_densenet = model_densenet.fit(
    train_generator,
    steps_per_epoch=num_train_samples//batch_size,
    validation_data=validation_generator,
    validation_steps=num_validation_samples//batch_size,
    verbose=1,
    epochs=1000,
    callbacks=[
        EarlyStopping(
        monitor='val_loss',
        min_delta=0,
        patience=15,
        verbose=1,
        mode='min',
        baseline=None,
        restore_best_weights=True,
        start_from_epoch=20
        ),
    ],
    class_weight = class_weights_1
    )

**Results & Visualization**

In [ ]:
# list all data in history
history = history_val_fmed_densenet
print(history.history.keys())

**summarize history for fmed score**

In [ ]:
# summarize history for fmed score
plt.plot(history.history['fmed'])
plt.plot(history.history['val_fmed'])
plt.title('DenseNet Model Fmed score vs. epochs')
plt.ylabel('Fmed')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

**Summarize history for sensitivity score**

In [ ]:
# summarize history for sensitivity score
plt.plot(history.history['sensitivity'])
plt.plot(history.history['val_sensitivity'])
plt.title('DenseNet Model Sensitivity score vs. epochs')
plt.ylabel('Sensitivity')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc='lower right')
plt.show()



**summarize history for specificity score**

In [ ]:
# summarize history for specificity score
plt.plot(history.history['specificity'])
plt.plot(history.history['val_specificity'])
plt.title('DenseNet Model Specificity score vs. epochs')
plt.ylabel('Specificity')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc='lower right')
plt.show()


**summarize history for f1 score**

In [ ]:
# summarize history for f1 score
plt.plot(history.history['f1'])
plt.plot(history.history['val_f1'])
plt.title('DenseNet Model F1 score vs. epochs')
plt.ylabel('F1')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc='lower right')
plt.show()



**summarize history for accuracy**

In [ ]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('DenseNet Model accuracy vs. epochs')
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

**Summarise history for loss**

In [ ]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('DenseNet Model loss vs. epochs')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc='lower right')
plt.show()

In [ ]:
test_generator = test_datagen.flow_from_directory(
        test_dir_path,
        target_size=(image_height, image_width),
        batch_size=8,
        color_mode="rgb",
        class_mode='binary',
        shuffle=True,
)

In [ ]:
results = model_densenet.evaluate(test_generator)
print(f"test_loss = {results[0]}")
print(f"test_accuracy = {results[1]}")
print(f"test_sensitivity = {results[2]}")
print(f"test_specificity = {results[3]}")
print(f"test_fmed = {results[4]}")
print(f"test_f1_score = {results[5]}")

In [ ]:
probabilities = model_densenet.predict(test_generator)
y_true = test_generator.classes
y_pred = probabilities > 0.5

In [ ]:
aa = confusion_matrix(y_true, y_pred)

In [ ]:
aa

In [ ]:
tn, fp, fn, tp = aa.ravel()

In [ ]:
specificity = tn / (tn+fp)
print(specificity)

sensitivity = tp / (tp + fn)
print(sensitivity)

In [ ]:
accuracy = (tn+tp) / (tn+fp+tp+fn)
accuracy
precision = tp / (tp+fp)
precision
recall = tp / (tp+fn)
recall
f1_score = 2* ((precision*recall)/(precision+recall))
f1_score

print("Accuracy   : " , accuracy)
print("Precision  : " , precision)
print("Recall     : " , recall)
print("F1 Score   : " , f1_score)